# Lesson 3.6: Retrieval-Augmented Generation (RAG)

---

In the previous lessons, we built the foundation for data processing with LangChain: loading documents, splitting them, creating embeddings, and storing them in a Vector Store. Now, we will combine all these components to create one of the most powerful and popular architectures in modern LLM applications: **Retrieval-Augmented Generation (RAG)**.

## 1. Concept of Retrieval-Augmented Generation (RAG)

### 1.1. What is RAG?

**Retrieval-Augmented Generation (RAG)** is a technique in the field of Large Language Models (LLMs) that combines the ability to **retrieve information** from an external knowledge base with the LLM's **text generation** capabilities.

Instead of solely relying on the knowledge pre-trained within the model (which can be outdated or incomplete), a RAG system will:
1.  **Retrieve** relevant text segments or documents from a database (typically a Vector Store) based on the user's query.
2.  Provide this retrieved information as **additional context** to the LLM.
3.  The LLM then uses both the original user query and the retrieved context to **generate** the answer.

### 1.2. Benefits of RAG

RAG offers several significant advantages, addressing inherent limitations of standalone LLMs:

* **Reduces Hallucination:** LLMs tend to "make up" information if they lack sufficient data or context. RAG provides factual, reliable information, helping the LLM generate evidence-based answers.
* **Increases Accuracy and Relevance:** By retrieving documents directly relevant to the query, RAG ensures that the LLM's answers are more accurate and pertinent to the user's needs.
* **Provides Attribution:** RAG allows you to cite the source documents the LLM used to generate its answer, increasing transparency and trustworthiness. This is crucial in applications requiring high accuracy (e.g., medical, legal).
* **Flexible Knowledge Updates:** LLMs only have knowledge up to their last training cutoff. RAG allows you to update the external knowledge base (Vector Store) independently without retraining the entire LLM, keeping the system up-to-date with the latest information.
* **Handles Private/Proprietary Information:** You can use RAG to give an LLM access to internal, private company data without having to retrain the LLM on that data.
* **Cost Reduction:** Instead of using very large (and expensive) LLMs with massive context windows or fine-tuning LLMs, RAG allows you to use smaller, more efficient LLMs by providing precise context.


---

## 2. Architecture of a Basic RAG System

A basic RAG system is typically divided into two main phases: **Indexing** and **Runtime**, with Runtime comprising **Retrieval** and **Generation**.

### 2.1. Indexing Phase

This phase occurs **before** a user asks a question. The goal is to prepare your knowledge base for efficient retrieval.

1.  **Document Loading:**
    * Use **Document Loaders** (Lesson 3.1) to load data from various sources (TXT files, PDFs, web pages, CSVs, etc.) into LangChain's `Document` objects.
2.  **Text Splitting:**
    * Use **Text Splitters** (Lesson 3.2) to break down large documents into smaller text segments (chunks) to fit within LLM token limits and optimize search.
3.  **Creating Embeddings:**
    * Use **Embedding models** (Lesson 3.3) to convert each text chunk into a numerical vector (embedding).
4.  **Storing in Vector Store:**
    * Store these embedding vectors along with the original `Document`s (or their metadata) in a **Vector Store** (Lesson 3.3, 3.4) like FAISS, Chroma, Pinecone, etc., for efficient searching.

### 2.2. Retrieval Phase

This phase occurs **when** a user asks a question.

1.  The user poses a **query**.
2.  This query is converted into an embedding vector using the same embedding model used in the Indexing phase.
3.  The **Retriever** (Lesson 3.5) uses this query vector to search for the most similar embedding vectors in the Vector Store.
4.  The Retriever returns the most relevant text segments (chunks).

### 2.3. Generation Phase

This phase also occurs **when** a user asks a question.

1.  The relevant text segments retrieved from the Retrieval phase are provided as **context** to the LLM.
2.  A **Prompt Template** is constructed, incorporating both the user's query and the retrieved context.
3.  The LLM receives this prompt and generates the final answer based on both its internal knowledge and the information provided in the context.


---

## 3. Building a Question Answering (Q&A) System on Your Own Documents

Now, we will build a simple Question Answering system using the RAG architecture on a custom document.

### 3.1. Environment and Data Preparation

**Preparation:**
* Ensure you have the necessary libraries installed: `langchain-openai`, `chromadb`, `pypdf`.
* Set the `OPENAI_API_KEY` environment variable.
* Prepare a sample PDF file (e.g., `sample_document.pdf`) in the same directory as your code. If you don't have a real PDF, you can create a dummy file or use a long text string.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai chromadb pypdf reportlab

import os
import shutil
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 1. Prepare sample document (PDF file)
# Create a dummy PDF file if a real one is not available for the code to run
pdf_file_path = "sample_document.pdf"
try:
    from reportlab.pdfgen import canvas
    c = canvas.Canvas(pdf_file_path)
    c.drawString(100, 750, "This is a sample document about LangChain and RAG.")
    c.drawString(100, 730, "LangChain is a framework for building LLM applications.")
    c.drawString(100, 710, "RAG combines information retrieval with LLM text generation.")
    c.drawString(100, 690, "It helps reduce 'hallucinations' and increase accuracy.")
    c.drawString(100, 670, "RAG steps include Loading, Splitting, Embedding, Storing, Retrieving, and Generating.")
    c.drawString(100, 650, "Chroma is a popular open-source Vector Store.")
    c.drawString(100, 630, "FAISS is also a good option for local Vector Stores.")
    c.save()
    print(f"Sample PDF file created: {pdf_file_path}")
except ImportError:
    with open(pdf_file_path, "w") as f:
        f.write("This is a dummy PDF file. Please replace with a real PDF.\n")
    print("Could not create real PDF with reportlab. Using dummy file.")
    print("Please ensure you have a real PDF file 'sample_document.pdf' for the example to work best.")

# Initialize LLM and Embeddings Model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Directory for ChromaDB persistence
persist_directory = "./chroma_db_rag_demo"
# Remove old directory if it exists to ensure a clean start
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
    print(f"Old Chroma directory removed: {persist_directory}")

### 3.2. Building the Indexing Phase

We will load the document, split it, and store it in a Chroma Vector Store.

In [ ]:
# --- Indexing Phase ---

# 1. Load documents
print("Loading documents...")
loader = PyPDFLoader(pdf_file_path)
documents = loader.load()
print(f"Loaded {len(documents)} pages from PDF.")

# 2. Split text
print("Splitting text...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    add_start_index=True
)
chunks = text_splitter.split_documents(documents)
print(f"Split into {len(chunks)} chunks.")

# 3. Create Embeddings and Store in Vector Store
print("Creating embeddings and storing in Chroma Vector Store...")
vector_store = Chroma.from_documents(
    chunks,
    embeddings,
    persist_directory=persist_directory
)
print("Indexing phase completed.")

# 4. Create Retriever from Vector Store
retriever = vector_store.as_retriever(search_kwargs={"k": 3}) # Retrieve 3 most relevant chunks
print(f"Retriever created, will retrieve k=3 documents.")

### 3.3. Building the Retrieval and Generation Phase

We will combine the Retriever with the LLM to create the RAG chain.

In [ ]:
# --- Retrieval and Generation Phase ---

# 1. Define Prompt for RAG
# This prompt will include the user's query and the retrieved context
rag_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful Q&A assistant. Answer the user's question based on the provided context. If you cannot find the answer in the context, state that you don't know."),
    HumanMessage(content="Context: {context}\n\nQuestion: {question}"),
])

# 2. Build the RAG chain using LCEL
# This chain will:
# - Take the user's question.
# - Retrieve context using the Retriever.
# - Format the prompt with the question and context.
# - Invoke the LLM to generate the answer.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} # Retriever will take the question as input
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("RAG chain built.")

### 3.4. Executing the Q&A System

Now, let's try asking a few questions to your RAG system.

In [ ]:
# --- Execute the Q&A System ---

print("\n--- Starting RAG Q&A System ---")

# Question 1
question_1 = "What is RAG and how does it help?"
print(f"\nQuestion: {question_1}")
answer_1 = rag_chain.invoke({"question": question_1})
print(f"Answer: {answer_1}")

# Question 2
question_2 = "What are the main steps to build a RAG system?"
print(f"\nQuestion: {question_2}")
answer_2 = rag_chain.invoke({"question": question_2})
print(f"Answer: {answer_2}")

# Question 3 (out of context)
question_3 = "What is the capital of France?"
print(f"\nQuestion: {question_3}")
answer_3 = rag_chain.invoke({"question": question_3})
print(f"Answer: {answer_3}") # LLM might answer if it has general knowledge, or say it doesn't know if the prompt is well-crafted.

# Clean up sample PDF file and Chroma directory
os.remove(pdf_file_path)
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
print(f"\nSample PDF file and Chroma directory '{persist_directory}' removed.")

print("\n--- RAG Q&A System Ended ---")

**Data Flow Explanation in `rag_chain`:**

1.  `rag_chain.invoke({"question": question_1})`: The input is a dictionary containing the `question` key.
2.  `{"context": retriever, "question": RunnablePassthrough()}`:
    * This is a `RunnableParallel`.
    * `retriever`: Receives the value of `question` from the `rag_chain`'s input and performs retrieval of relevant documents from the Vector Store. The result is a list of `Document` objects. This list is assigned to the `context` key.
    * `RunnablePassthrough()`: Receives the entire original input dictionary (`{"question": "..."}`) and passes it through. This result is assigned to the `question` key in the `RunnableParallel`'s output.
    * The output of this step is a dictionary: `{"context": [doc1, doc2, ...], "question": "..."}`.
3.  `| rag_prompt`:
    * `rag_prompt` receives this dictionary. It will take the list of `Document`s from the `context` key and the `question` from the `question` key.
    * It formats the prompt according to the defined structure, inserting the content of the `Document`s into the `{context}` placeholder and the question into the `{question}`.
    * The result is a `ChatPromptValue` object (a list of messages).
4.  `| llm`:
    * `llm` receives the `ChatPromptValue` object and sends it to the LLM model (e.g., GPT-3.5 Turbo).
    * The LLM processes the prompt and generates a response. The result is an `AIMessage` object.
5.  `| StrOutputParser()`:
    * `StrOutputParser` receives the `AIMessage` object and extracts the string content of the LLM's response.
    * This is the final output of the `rag_chain`.


---

## Lesson Summary

This lesson provided a comprehensive introduction to **Retrieval-Augmented Generation (RAG)**, a powerful architecture that combines information retrieval capabilities with LLM text generation. We learned about the **benefits of RAG**, including reducing "hallucinations," increasing accuracy, and providing attribution. The lesson also delved into the **architecture of a basic RAG system**, divided into two main phases: **Indexing** (loading, splitting, embedding, and storing documents) and **Runtime** (comprising **Retrieval** for fetching relevant text segments and **Generation** for the LLM to produce answers based on the retrieved context). Finally, you practiced **building a Question Answering (Q&A) system on your own documents**, integrating all the learned concepts about Document Loaders, Text Splitters, Embeddings, Vector Stores, and Retrievers to create an LLM application capable of querying specialized knowledge.